In [1]:
from window_generator import WindowGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
from read_data import *
from methods import *
MAX_EPOCHS = 20
VOICE = 2

In [12]:
def compile_and_fit(model, window, patience=2, verbose=0):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min',
                                                     restore_best_weights=True)
#  Early stopping when there is no improvement in loss. 
    model.compile(loss=tf.losses.CategoricalCrossentropy(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.CategoricalAccuracy(), tf.metrics.CategoricalCrossentropy(),tf.metrics.MeanAbsoluteError()])

# At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metrics.
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                        verbose=verbose)
    return history

In [3]:
multi_step_window = WindowGenerator(input_width=54, label_width=1, shift=1, batch_size=1, voice_number=VOICE)

for example_inputs, example_labels in multi_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

print(multi_step_window.train)
# print(len(multi_step_window.label_columns_indices))

Inputs shape (batch, time, features): (1, 54, 6)
Labels shape (batch, time, features): (1, 1, 23)
<TakeDataset shapes: ((None, 54, 6), (None, 1, 23)), types: (tf.float32, tf.float32)>


In [14]:
n_output_nodes = len(multi_step_window.label_columns_indices)

initializer = tf.keras.initializers.RandomUniform(minval=-0.50, maxval=0.50)
linear_multi_step = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, kernel_initializer=initializer, activation='relu'),
    tf.keras.layers.Dense(units=64, kernel_initializer=initializer, activation='relu'),
    tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear_multi_step(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(linear_multi_step, multi_step_window, verbose=2)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = linear_multi_step.evaluate(multi_step_window.test)
dict(zip(linear_multi_step.metrics_names, result))

Input shape: (1, 54, 6)
Output shape: (1, 1, 23)
Epoch 1/20
2637/2637 - 4s - loss: 2.7540 - categorical_accuracy: 0.4293 - categorical_crossentropy: 2.7540 - mean_absolute_error: 0.0548 - val_loss: 1.5018 - val_categorical_accuracy: 0.6109 - val_categorical_crossentropy: 1.5018 - val_mean_absolute_error: 0.0453
Epoch 2/20
2637/2637 - 4s - loss: 1.2850 - categorical_accuracy: 0.6526 - categorical_crossentropy: 1.2850 - mean_absolute_error: 0.0391 - val_loss: 0.9933 - val_categorical_accuracy: 0.7437 - val_categorical_crossentropy: 0.9933 - val_mean_absolute_error: 0.0331
Epoch 3/20
2637/2637 - 3s - loss: 0.8761 - categorical_accuracy: 0.7584 - categorical_crossentropy: 0.8761 - mean_absolute_error: 0.0298 - val_loss: 0.6893 - val_categorical_accuracy: 0.8021 - val_categorical_crossentropy: 0.6893 - val_mean_absolute_error: 0.0251
Epoch 4/20
2637/2637 - 3s - loss: 0.6286 - categorical_accuracy: 0.8271 - categorical_crossentropy: 0.6286 - mean_absolute_error: 0.0225 - val_loss: 0.6213 - v

{'loss': 0.3811323344707489,
 'categorical_accuracy': 0.9095744490623474,
 'categorical_crossentropy': 0.3811323344707489,
 'mean_absolute_error': 0.012468373402953148}

In [19]:
n_output_nodes = len(multi_step_window.label_columns_indices)

rnn = tf.keras.Sequential([
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', rnn(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(rnn, multi_step_window, verbose=1)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = rnn.evaluate(multi_step_window.test)
dict(zip(rnn.metrics_names, result))

Input shape: (1, 54, 6)
Output shape: (1, 1, 23)
Epoch 1/20
2637/2637 [==============================] - 40s 15ms/step - loss: 1.7385 - categorical_accuracy: 0.5157 - categorical_crossentropy: 1.7385 - mean_absolute_error: 0.0575 - val_loss: 0.9773 - val_categorical_accuracy: 0.7543 - val_categorical_crossentropy: 0.9773 - val_mean_absolute_error: 0.0378
Epoch 2/20
2637/2637 [==============================] - 38s 15ms/step - loss: 0.9402 - categorical_accuracy: 0.7617 - categorical_crossentropy: 0.9402 - mean_absolute_error: 0.0355 - val_loss: 0.8149 - val_categorical_accuracy: 0.8074 - val_categorical_crossentropy: 0.8149 - val_mean_absolute_error: 0.0315
Epoch 3/20
2637/2637 [==============================] - 40s 15ms/step - loss: 0.8233 - categorical_accuracy: 0.7957 - categorical_crossentropy: 0.8233 - mean_absolute_error: 0.0309 - val_loss: 0.7422 - val_categorical_accuracy: 0.8167 - val_categorical_crossentropy: 0.7422 - val_mean_absolute_error: 0.0286
Epoch 4/20
2637/2637 [=====

{'loss': 0.39688539505004883,
 'categorical_accuracy': 0.8856382966041565,
 'categorical_crossentropy': 0.39688539505004883,
 'mean_absolute_error': 0.014796498231589794}

In [6]:
# Get last window
def predict_new_pitches(model, n=100):
    last_window = None
    last_pitch = None
    last_duration = None
    all_predicted_pitches = []
    all_probs = []

    for i in range(n):
        if last_window is None:
            # Use an iterator to get the last window of the dataset. 
            for last_window, labels in multi_step_window.test_no_shuffle.as_numpy_iterator():
                pass

#             print('window:', type(last_window),last_window ,'\n====\nlabels', labels)
#             print('window shape:', last_window.shape, ' label shape:', labels.shape)
        
        # Model is expecting a shape of (batch, timestep, features) but for one sample this is (6,6)
        # Add dimension to input
        probabilities = model.predict(np.array([last_window]))[0]
        probabilities = probabilities[0]
        all_probs.append(probabilities)
        
        predicted_pitch = get_pitch_from_probability(
            probabilities,
            key=multi_step_window.pitch_conversion_key,
            method=SelectionMethod.PROB, n=10,
        )
        all_predicted_pitches.append(predicted_pitch)
        features = get_pitch_features(predicted_pitch)

        if last_pitch is None:
            last_pitch = get_voice(VOICE)[-1]

        # - find new duration based on previous and current pitch
        if last_pitch == predicted_pitch:
            if last_duration is None:
                duration = multi_step_window.df.iloc[-1]["dur"] + 1   # bug .
            else:
                duration = last_duration + 1
        else:
            duration = 1

        last_duration = duration
        last_pitch = predicted_pitch

        # create new dataframe
        new_df = pd.DataFrame(data={"dur": duration}, dtype=float, index=[0])
        (
            new_df["log_pitch"],
            new_df["chroma_x"],
            new_df["chroma_y"],
            new_df["c5_x"],
            new_df["c5_y"],
        ) = features
        # - Normalise features
        new_df = (new_df - multi_step_window.mean_df) / multi_step_window.std_df
        # - Make new window based on previous timesteps with a slide of 1 and the normalised features
        last_window = np.append(last_window[1:], new_df.values.tolist(), axis=0)
    return all_predicted_pitches, all_probs

In [20]:
predictions, probs = predict_new_pitches(rnn, n=400)
print(predictions)
print(probs[-1])
sum(probs[-1])
# wg = WindowGenerator(input_width=6, label_width=1, shift=1, batch_size=1)
# print(f'train {sum(1 for _ in wg.train)} val {sum(1 for _ in wg.val)} test {sum(1 for _ in wg.test)} ')
# print(f'full dataset {sum(1 for _ in wg.full_dataset)}')
# print(f'data size {wg.df.shape[0]} | file voice size: {len(get_voice(0))}')

# max(get_voice(2))



[54, 54, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 50, 50, 49, 49, 47, 47, 50, 50, 50, 50, 50, 50, 49, 49, 47, 47, 49, 52, 52, 52, 52, 52, 52, 52, 52, 50, 50, 50, 50, 49, 49, 47, 47, 45, 45, 49, 49, 50, 50, 52, 52, 50, 50, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 50, 50, 50, 50, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,

0.00023248768951589227